# Initializing the Guide to Pharmacology Database

---



## Setup and Data Loading

First, let's review the setup and data loading process:

In [1]:
!pip install -q rdflib-neo4j openpyxl


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from rdflib_neo4j import Neo4jStoreConfig
from rdflib_neo4j import HANDLE_VOCAB_URI_STRATEGY
# from google.colab import userdata

In [3]:
import os

# NEO_DB_URI = 'bolt://172.18.176.1:7687'
NEO_DB_URI = os.getenv('NEO4J_LCL_URI')
NEO_DB_USERNAME = os.getenv('NEO4J_USERNAME')
NEO_DB_PWD = os.getenv('NEO4J_LCL_PASSWORD')


In [4]:
auth_data = {'uri': NEO_DB_URI,
             'database': "neo4j",
             'user': NEO_DB_USERNAME,
             'pwd': NEO_DB_PWD}

## Define namespaces and configuration


In [5]:
from rdflib import Namespace

prefixes = {
    'gtpo': Namespace('https://rdf.guidetopharmacology.org/ns/gtpo#'),
    'grac': Namespace('https://rdf.guidetopharmacology.org/GRAC/'),
    'cito': Namespace('http://purl.org/spar/cito/'),
    'dcat': Namespace('http://www.w3.org/ns/dcat#'),
    'dctypes': Namespace('http://purl.org/dc/dcmitype/'),
    'dct': Namespace('http://purl.org/dc/terms/'),
    'foaf': Namespace('http://xmlns.com/foaf/0.1/'),
    'freq': Namespace('http://purl.org/cld/freq/'),
    'idot': Namespace('http://identifiers.org/idot/'),
    'lexvo': Namespace('http://lexvo.org/ontology#'),
    'pav': Namespace('http://purl.org/pav/'),
    'prov': Namespace('http://www.w3.org/ns/prov#'),
    'rdf': Namespace('http://www.w3.org/1999/02/22-rdf-syntax-ns#'),
    'rdfs': Namespace('http://www.w3.org/2000/01/rdf-schema#'),
    'schemaorg': Namespace('http://schema.org/'),
    'sd': Namespace('http://www.w3.org/ns/sparql-service-description#'),
    'sio': Namespace('http://semanticscience.org/resource/'),
    'void': Namespace('http://rdfs.org/ns/void#'),
    'void-ext': Namespace('http://ldf.fi/void-ext#'),
    'xsd': Namespace('http://www.w3.org/2001/XMLSchema#'),
    # Add other required prefixes based on your data inspection
}

In [6]:
config = Neo4jStoreConfig(auth_data=auth_data,
                          custom_prefixes=prefixes,
                          handle_vocab_uri_strategy=HANDLE_VOCAB_URI_STRATEGY.IGNORE,
                          batching=True)

## Load the ontology data

In [7]:
from rdflib_neo4j import Neo4jStore
from rdflib import Graph
file_path = 'https://www.guidetopharmacology.org/DATA/rdf/2024.3/gtp-rdf.n3'

graph_store = Graph(store=Neo4jStore(config=config))
graph_store.parse(file_path,format="nt")
graph_store.close(True)

Uniqueness constraint on :Resource(uri) found. 
                
                
The store is now: Open
The store is now: Closed
IMPORTED 387357 TRIPLES


This setup process installs the necessary libraries, defines authentication data, sets up namespaces, and loads the Guide to Pharmacology ontology data into a Neo4j graph database.

## Querying the Guide to Pharmacology data

Now, let's explore the GtoP database data using a Cypher query:

In [8]:
from neo4j import GraphDatabase

# Initialize Neo4j driver
driver = GraphDatabase.driver(
    NEO_DB_URI,
    auth=(NEO_DB_USERNAME, NEO_DB_PWD)
)


#### Setup reused functions

In [9]:
def run_cypher_query(query):
    with driver.session(database="neo4j") as session:
        result = session.run(query)
        # Fetch all results and convert them into a list of dictionaries
        return [record.data() for record in result]

In [10]:
cypher_query = """
WITH ["EnzymeTargetFamily",
      "OtherProteinTargetFamily",
      "TransporterTargetFamily",
      "GProteinCoupledReceptorTargetFamily",
      "CatalyticReceptorTargetFamily",
      "LigandGatedIonChannelTargetFamily",
      "VoltageGatedIonChannelTargetFamily",
      "NuclearHormoneReceptorTargetFamily",
      "OtherIonChannelTargetFamily",
      "GroupingTargetFamily",
      "LigandTargetFamily"] AS targetFamilyTypes,

     ["SyntheticOrganicLigand",
      "NaturalProductLigand",
      "PeptideLigand",
      "MetaboliteLigand",
      "AntibodyLigand",
      "InorganicLigand"] AS ligandTypes,

     ["AntagonistInteraction",
      "AgonistInteraction",
      "ActivatorInteraction",
      "Interaction",
      "InhibitorInteraction",
      "AntibodyInteraction",
      "ChannelBlockerInteraction",
      "AllostericModulatorInteraction",
      "GatingInhibitorInteraction",
      "SubunitSpecificInteraction"] AS interactionTypes

UNWIND targetFamilyTypes AS targetFamilyType
UNWIND ligandTypes AS ligandType
UNWIND interactionTypes AS interactionType

CALL apoc.cypher.run("
    MATCH (tf:`" + targetFamilyType + "` {label: $label})<-[r1:hasTargetFamily]-(t:Target)
    MATCH (i:`" + interactionType + "`)-[r2:hasTarget]->(t)
    MATCH (i)-[r3:hasLigand]->(l:`" + ligandType + "`)
    RETURN tf, r1, t, r2, i, r3, l
    LIMIT 30
", {label: "STE7 family"}) YIELD value

RETURN value.tf AS tf, value.r1 AS r1, value.t AS t, value.r2 AS r2, value.i AS i, value.r3 AS r3, value.l AS l
"""

In [11]:
from pprint import pprint

# Execute the query and display the results
results = run_cypher_query(cypher_query)
pprint(results)

[{'i': {'uri': 'https://rdf.guidetopharmacology.org/GRAC/interaction88688'},
  'l': {'approved': 'f',
        'canonicalSMILES': 'COc1ccc(cc1)[C@H]1CC(=O)c2c1c(OC)cc(c2)OC',
        'comment': '(R)-STU104 is a first-in-class molecule that inhibits the '
                   'interaction between the proteins kinases TAK1 '
                   '(mitogen-activated protein kinase kinase kinase 7; MAP3K7) '
                   'and MKK3 (mitogen-activated protein kinase kinase 3; '
                   'MAP2K3) <Reference id=43711/>. These kinases are part of '
                   'the inflammatory signalling cascade that culminates in '
                   'TNF-&alpha; production. (R)-STU104 binds to MKK3 at a '
                   'location that disrupts its interaction with the upstream '
                   'kinase TAK1. This action perturbs MKK3 phosphorylation by '
                   'TAK1 and inhibits downstream signal propagation.  Blocking '
                   'TAK1-mediated phosphorylation 

This query retrieves information about the STE7 target family, including related targets, interactions, and ligands.

### Querying for a list of drug compounds

In [12]:
cypher_query = """
MATCH (l:Resource)
WHERE l.ligandName IN ['RGFP966', 'belinostat', 'ricolinostat', 'nexturastat A', 'pyroxamide', 'entinostat', 'resminostat', 'panobinostat', 'R306465', 'M 344', 'AR-42']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
RETURN path, a, af, r
"""

In [13]:
# Execute the query and display the results
results = run_cypher_query(cypher_query)

In [14]:
from pprint import pprint

pprint(results)

[{'a': {'uri': 'https://rdf.guidetopharmacology.org/ns/gtpo#InhibitionAction'},
  'af': {'hasMedianValue': 7.099999904632568,
         'uri': 'https://rdf.guidetopharmacology.org/GRAC/affinity79868'},
  'path': [{'approved': 'f',
            'canonicalSMILES': 'O=C(Nc1ccc(cc1N)F)C=Cc1cnn(c1)CC=Cc1ccccc1',
            'comment': 'RGFP966 is a selective inhibitor of histone '
                       'deacetylase 3 (HDAC3) <Reference id=24589/>. In rodent '
                       'studies, RGFP966-induced inhibition of HDAC3 appears '
                       'to enhance memory processes involved in extinction of '
                       'drug-seeking behaviour <Reference id=24589/>.',
            'inChI': 'InChI=1S/C21H19FN4O/c22-18-9-10-20(19(23)13-18)25-21(27)11-8-17-14-24-26(15-17)12-4-7-16-5-2-1-3-6-16/h1-11,13-15H,12,23H2,(H,25,27)/b7-4+,11-8+',
            'inChIKey': 'BLVQHYHDYFTPDV-VCABWLAWSA-N',
            'isomericSMILES': 'O=C(Nc1ccc(cc1N)F)/C=C/c1cnn(c1)C/C=C/c1ccccc1',
       

### Querying for lists of compounds - tabular format

In [23]:
cypher_query_hdac = """
// HDAC Inhibitors
MATCH (l:Resource)
WHERE l.ligandName IN ['RGFP966', 'belinostat', 'ricolinostat', 'nexturastat A', 'pyroxamide', '5-nitroso-8-quinolinol', 'entinostat', 'resminostat', 'panobinostat', 'R306465', 'M 344', 'AR-42']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
    OPTIONAL MATCH (af)-[:hasUnits]-(afu)
RETURN 
    l.label as Compound,
    l.approved as FDA_Approved,
    t.label as Target,
    tf.label as Target_Family,
    a.uri as Action,
    af.hasMedianValue as Affinity_Median,
    af.hasLowValue as Affinity_Low,
    af.hasHighValue as Affinity_High,
    afu.uri as Affinity_Units,
    r.uri as Reference,
    l.comment as Description,
    l.inChIKey as InChIKey,
    l.canonicalSMILES as SMILES,
    xr.uri as CHEMBL
ORDER BY l.ligandName, t.nomenclature
"""

cypher_query_cdk = """
// CDK Inhibitors
MATCH (l:Resource)
WHERE l.ligandName IN ['indisulam', 'dinaciclib', 'flavopiridol', 'Bms-265246', 'AT7519', 'PHA-793887', 'riviciclib', 'R-547', '7-hydroxystaurosporine', 'SB1317/TG02', 'Sns-032', 'AZD5438']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
    OPTIONAL MATCH (af)-[:hasUnits]-(afu)
RETURN 
    l.label as Compound,
    l.approved as FDA_Approved,
    t.label as Target,
    tf.label as Target_Family,
    a.uri as Action,
    af.hasMedianValue as Affinity_Median,
    af.hasLowValue as Affinity_Low,
    af.hasHighValue as Affinity_High,
    afu.uri as Affinity_Units,
    r.uri as Reference,
    l.comment as Description,
    l.inChIKey as InChIKey,
    l.canonicalSMILES as SMILES,
    xr.uri as CHEMBL
ORDER BY l.ligandName, t.nomenclature
"""

cypher_query_aurk = """
// AURK Inhibitors
MATCH (l:Resource)
WHERE l.ligandName IN ['AZD1152-HQPA', 'tozasertib', 'SNS-314 mesylate', 'AMG 900', 'ZM-447439', 'ENMD-2076', 'CYC116', 'alisertib', 'PF-03814735', 'CCT129202', 'CCT137690']
MATCH path = (l)-[:hasLigand]-(i)-[:hasTarget]-(t)-[:hasTargetFamily]-(tf)
    OPTIONAL MATCH (i)-[:hasAction]-(a)
    OPTIONAL MATCH (i)-[:hasAffinity]-(af)
    OPTIONAL MATCH (i)-[:hasReference]-(r)
    OPTIONAL MATCH (l)-[:xref]-(xr)
    OPTIONAL MATCH (af)-[:hasUnits]-(afu)
RETURN 
    l.label as Compound,
    l.approved as FDA_Approved,
    t.label as Target,
    tf.label as Target_Family,
    a.uri as Action,
    af.hasMedianValue as Affinity_Median,
    af.hasLowValue as Affinity_Low,
    af.hasHighValue as Affinity_High,
    afu.uri as Affinity_Units,
    r.uri as Reference,
    l.comment as Description,
    l.inChIKey as InChIKey,
    l.canonicalSMILES as SMILES,
    xr.uri as CHEMBL
ORDER BY l.ligandName, t.nomenclature
"""

In [24]:
import pandas as pd

# Execute the query and display the results
results_hdac = run_cypher_query(cypher_query_hdac)
results_cdk = run_cypher_query(cypher_query_cdk)
results_aurk = run_cypher_query(cypher_query_aurk)

df_hdac = pd.DataFrame(results_hdac)
df_cdk = pd.DataFrame(results_cdk)
df_aurk = pd.DataFrame(results_aurk)

/tmp/ipykernel_58623/4134650490.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:


In [25]:
df_hdac

,Compound,FDA_Approved,Target,Target_Family,Action,Affinity_Median,Affinity_Low,Affinity_High,Affinity_Units,Reference,Description,InChIKey,SMILES,CHEMBL
0,RGFP966,f,histone deacetylase 3,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.10,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/23297220,RGFP966 is a selective inhibitor of histone de...,BLVQHYHDYFTPDV-VCABWLAWSA-N,O=C(Nc1ccc(cc1N)F)C=Cc1cnn(c1)CC=Cc1ccccc1,None
1,belinostat,t,histone deacetylase 1,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.39,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/17868033,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
2,belinostat,t,histone deacetylase 1,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,9.07,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/20139990,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
3,belinostat,t,histone deacetylase 2,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,6.90,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/17868033,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
4,belinostat,t,histone deacetylase 2,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,9.07,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/20139990,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
5,belinostat,t,histone deacetylase 3,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,8.82,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/20139990,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
6,belinostat,t,histone deacetylase 3,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.52,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/17868033,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
7,belinostat,t,histone deacetylase 4,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,6.94,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/17868033,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
8,belinostat,t,histone deacetylase 4,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,6.42,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/20139990,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)Nc1ccccc1,http://identifiers.org/chembl.compound/CHEMBL4...
9,belinostat,t,histone deacetylase 5,3.5.1.- Histone deacetylases (HDACs),https://rdf.guidetopharmacology.org/ns/gtpo#In...,6.76,None,None,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/20139990,Belinostat is a pan-histone deacetylase (HDAC)...,NCNRHFGMJRPRSK-MDZDMXLPSA-N,ONC(=O)C=Cc1cccc(c1)S(=O)(=O)

In [26]:
df_cdk

,Compound,FDA_Approved,Target,Target_Family,Action,Affinity_Median,Affinity_Low,Affinity_High,Affinity_Units,Reference,Description,InChIKey,SMILES,CHEMBL
0,7-hydroxystaurosporine,f,3-phosphoinositide dependent protein kinase 1,PDK1 family,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.48,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/11896604,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
1,7-hydroxystaurosporine,f,"LCK proto-oncogene, Src family tyrosine kinase",Src family,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.30,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
2,7-hydroxystaurosporine,f,checkpoint kinase 1,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,NaN,7.46,8.15,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
3,7-hydroxystaurosporine,f,checkpoint kinase 1,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,NaN,7.46,8.15,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
4,7-hydroxystaurosporine,f,checkpoint kinase 1,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,NaN,7.46,8.15,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/17292828,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
5,7-hydroxystaurosporine,f,checkpoint kinase 2,RAD53 family,https://rdf.guidetopharmacology.org/ns/gtpo#In...,5.98,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
6,7-hydroxystaurosporine,f,checkpoint kinase 2,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,5.98,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
7,7-hydroxystaurosporine,f,cyclin dependent kinase 1,CDK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.30,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
8,7-hydroxystaurosporine,f,glycogen synthase kinase 3 beta,GSK subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,6.30,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
9,7-hydroxystaurosporine,f,microtubule affinity regulating kinase 1,MARK subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.57,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosp

In [27]:
df_cdk

,Compound,FDA_Approved,Target,Target_Family,Action,Affinity_Median,Affinity_Low,Affinity_High,Affinity_Units,Reference,Description,InChIKey,SMILES,CHEMBL
0,7-hydroxystaurosporine,f,3-phosphoinositide dependent protein kinase 1,PDK1 family,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.48,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/11896604,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
1,7-hydroxystaurosporine,f,"LCK proto-oncogene, Src family tyrosine kinase",Src family,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.30,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
2,7-hydroxystaurosporine,f,checkpoint kinase 1,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,NaN,7.46,8.15,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
3,7-hydroxystaurosporine,f,checkpoint kinase 1,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,NaN,7.46,8.15,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
4,7-hydroxystaurosporine,f,checkpoint kinase 1,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,NaN,7.46,8.15,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/17292828,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
5,7-hydroxystaurosporine,f,checkpoint kinase 2,RAD53 family,https://rdf.guidetopharmacology.org/ns/gtpo#In...,5.98,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
6,7-hydroxystaurosporine,f,checkpoint kinase 2,CHK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,5.98,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
7,7-hydroxystaurosporine,f,cyclin dependent kinase 1,CDK1 subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.30,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
8,7-hydroxystaurosporine,f,glycogen synthase kinase 3 beta,GSK subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,6.30,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/15486189,7-hydroxystaurosporine is a cell-permeable <Li...,PBCZSGKMGDDXIJ-HQCWYSJUSA-N,CNC1CC2OC(C1OC)(C)n1c3ccccc3c3c1c1n2c2ccccc2c1...,http://identifiers.org/chembl.compound/CHEMBL1...
9,7-hydroxystaurosporine,f,microtubule affinity regulating kinase 1,MARK subfamily,https://rdf.guidetopharmacology.org/ns/gtpo#In...,7.57,NaN,NaN,http://www.bioassayontology.org/bao/bao_comple...,http://identifiers.org/pubmed/10786669,7-hydroxystaurosp

In [28]:
# export each DataFrame to a xlsx file
df_hdac.to_excel('hdac_inhibitors_data.xlsx', index=False)
df_cdk.to_excel('cdk_inhibitors_data.xlsx', index=False)
df_aurk.to_excel('aurk_inhibitors_data.xlsx', index=False)

In [29]:
driver.close()

### NOTES:  additional tools

#### Deleting Neo4j database nodes and relationships

In [30]:
# uncomment and use only when required - a quick way to clear the database

# cypher_query = """
# MATCH (n) DETACH DELETE n
# """

# results = run_cypher_query(cypher_query)
# driver.close()


## Citation

**BibTeX:**

```
@article{10.1093/nar/gkad944,
    author = {Harding, Simon D and Armstrong, Jane F and Faccenda, Elena and Southan, Christopher and Alexander, Stephen P H and Davenport, Anthony P and Spedding, Michael and Davies, Jamie A},
    title = "{The IUPHAR/BPS Guide to PHARMACOLOGY in 2024}",
    journal = {Nucleic Acids Research},
    volume = {52},
    number = {D1},
    pages = {D1438-D1449},
    year = {2023},
    month = {10},
    abstract = "{The IUPHAR/BPS Guide to PHARMACOLOGY (GtoPdb; https://www.guidetopharmacology.org) is an open-access, expert-curated, online database that provides succinct overviews and key references for pharmacological targets and their recommended experimental ligands. It includes over 3039 protein targets and 12 163 ligand molecules, including approved drugs, small molecules, peptides and antibodies. Here, we report recent developments to the resource and describe expansion in content over the six database releases made during the last two years. The database update section of this paper focuses on two areas relating to important global health challenges. The first, SARS-CoV-2 COVID-19, remains a major concern and we describe our efforts to expand the database to include a new family of coronavirus proteins. The second area is antimicrobial resistance, for which we have extended our coverage of antibacterials in partnership with AntibioticDB, a collaboration that has continued through support from GARDP. We discuss other areas of curation and also focus on our external links to resources such as PubChem that bring important synergies to the resources.}",
    issn = {0305-1048},
    doi = {10.1093/nar/gkad944},
    url = {https://doi.org/10.1093/nar/gkad944},
    eprint = {https://academic.oup.com/nar/article-pdf/52/D1/D1438/55039511/gkad944.pdf},
}
```